In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 844.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 3.2 MB/s eta 0:00:00


In [6]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [8]:
#load the model
model="meta-llama/Llama-2-7b-chat-hf"

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model,use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:757: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [10]:
model=AutoModelForCausalLM.from_pretrained(model,
                                           device_map='auto',
                                           torch_dtype=torch.float16,
                                           use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [11]:
pipeline=transformers.pipeline("text-generation",
                               model=model,
                               tokenizer=tokenizer,
                               torch_dtype=torch.bfloat16,
                               device_map="auto",
                               # Maximum number of tokens to generate, a word is generally 2-3 tokens (minimum: 1)
                               max_new_tokens=512,
                               #Minimum number of tokens to generate. To disable this set it to -1. A word is generally 2-3 tokens (minimum 1)
                               min_new_tokens=-1,
                               #Adjusts the randomness of output
                               temperature=0.75,
                               do_sample=True,
                               top_k=30,
                               num_return_sequences=1,
                               eos_token_id=tokenizer.eos_token_id)

In [12]:
llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [13]:
#https://replicate.com/meta/llama-2-7b-chat
llm.predict('please provide a summary of rich dad poor dad')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'please provide a summary of rich dad poor dad.\nThe following is a summary of the book "Rich Dad Poor Dad" by Robert Kiyosaki:\nThe book is written as a personal finance guide, but with a unique twist. The author, Robert Kiyosaki, shares his experiences and lessons learned from his two fathers - his own "poor dad," who is a well-educated but financially struggling government worker, and his friend\'s "rich dad," who is a successful businessman and investor.\n\nThe main themes of the book include:\n\n1. Financial education: The author argues that traditional education does not teach financial literacy and that individuals should learn how to manage their finances and invest their money wisely.\n2. Asset ownership: Rich dad teaches Robert that the key to financial freedom is owning assets that generate passive income, such as real estate, stocks, and businesses.\n3. Mindset: The book emphasizes the importance of having a growth mindset and being open to new ideas and opportunities.\n4. 

In [14]:
#Instruction as a prompt
B_INST, E_INST = "[INST]", "[/INST]"  #Input prompt
B_SYS, E_SYS = "<<SYS>>\n","\n<<SYS>>\n\n" #default prompt

In [15]:
#create our default system prompt
DEFAULT_SYSTEM_PROMPT="""\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
"""

In [16]:
#give instruction
instruction = "Who won the cricket mens T20 cricket world cup 2022?"

In [17]:
#Template
#2 things needed - 1. instruction and 2. system prompt

SYSTEM_PROMPT = B_SYS + DEFAULT_SYSTEM_PROMPT + E_SYS

In [18]:
SYSTEM_PROMPT

"<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\n"

In [19]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [20]:
template

"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\nWho won the cricket mens T20 cricket world cup 2022?[/INST]"

In [68]:
#Text translation - Prompts
instruction = "Convert the following text from English to German: \n\n{text}"

In [69]:
#System prompt
SYSTEM_PROMPT = B_SYS + DEFAULT_SYSTEM_PROMPT + E_SYS

In [70]:
SYSTEM_PROMPT

"<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\n"

In [71]:
template1 = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [72]:
template1

"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\nConvert the following text from English to German: \n\n{text}[/INST]"

In [73]:
print(template1)

In [74]:
#user text
prompt = PromptTemplate(input_variables=['text'],template=template1)

In [75]:
text = 'how are you?'

In [76]:
#LLM Chain
LLM_Chain = LLMChain(llm=llm, prompt = prompt)

In [77]:
LLM_Chain.run(text)

'[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\'t know the answer to a question, please don\'t share false information.\n\n<<SYS>>\n\nConvert the following text from English to German: \n\nhow are you?[/INST]  Hello! I\'m here to help you with your question. However, I must inform you that the question "how are you?" is not grammatically correct in German. The correct phrase in German would be "Wie geht es Ihnen?" (pronounced "vee-eh gah-tes IH-zzen").\n\nSo, to answer your question in a safe and respectful manner, I would say: "Wie geht es Ihnen?" (How are you?)\n\nPlease let me know 

In [78]:
#in the above result it gives me some extra output
#Custom prompt

instruction = "Convert the following text from English to German: \n\n{text}"


In [79]:
CUSTOM_SYSTEM_PROMPT = "You are an advance assistant that excels at translation"

In [80]:
SYSTEM_PROMPT = B_SYS + CUSTOM_SYSTEM_PROMPT + E_SYS

In [81]:
SYSTEM_PROMPT

'<<SYS>>\nYou are an advance assistant that excels at translation\n<<SYS>>\n\n'

In [82]:
template2 = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [83]:
template2

'[INST]<<SYS>>\nYou are an advance assistant that excels at translation\n<<SYS>>\n\nConvert the following text from English to German: \n\n{text}[/INST]'

In [84]:
print(template2)

In [86]:
prompt = PromptTemplate(input_variables=['text'],template=template2)

In [87]:
text = "How are you?"

In [88]:
#LLM Chain
LLM_Chain = LLMChain(llm=llm, prompt = prompt)

In [89]:
LLM_Chain.run(text)

'[INST]<<SYS>>\nYou are an advance assistant that excels at translation\n<<SYS>>\n\nConvert the following text from English to German: \n\nHow are you?[/INST]  Sure! Here\'s the translation of "How are you?" from English to German:\n\n"Wie geht es Ihnen?" (pronounced "vee-gret is-loh")\n\nI hope that helps! Let me know if you have any other questions.'

In [90]:
print(LLM_Chain.run(text))